# Coinbase Product Scanner
This notebook demonstrates how to fetch and process product data from Coinbase and map it to CoinGecko data.

## Step 1: Import Required Libraries
We start by importing the necessary libraries for data fetching and processing.

In [ ]:
import requests
import pandas as pd
import source.code.coinbase as cb

def get_products():
    return pd.DataFrame(cb.client.get_products(product_type='SPOT').to_dict()['products'])

product = get_products()
product = product.loc[product['quote_currency_id'] == 'USD']
product

/home/brian/Documents/cdp_api_key.json


,product_id,price,price_percentage_change_24h,volume_24h,volume_percentage_change_24h,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,...,alias,alias_to,base_display_symbol,quote_display_symbol,view_only,price_increment,display_name,product_venue,approximate_quote_24h_volume,new_at
0,BTC-USD,94553.87,0.84104709996005,5249.94756418,20.22667813660833,0.00000001,0.01,1,150000000,0.00000001,...,,[BTC-USDC],BTC,USD,False,0.01,BTC-USD,CBE,496402859.49,2023-01-01T00:00:00Z
2,ETH-USD,1769.56,-1.74678793128338,82779.39826387,22.35074028409949,0.00000001,0.01,1,150000000,0.00000001,...,,[ETH-USDC],ETH,USD,False,0.01,ETH-USD,CBE,146483111.99,2023-01-01T00:00:00Z
5,XRP-USD,2.1106,-0.19860034045773,63805432.601214,26.13049942744275,0.000001,0.0001,1,10000000,0.000001,...,,[XRP-USDC],XRP,USD,False,0.0001,XRP-USD,CBE,134667746.05,2023-07-13T19:47:00.692Z
6,SUI-USD,3.2291,-5.40207997656364,28404751.4,19.0323407770733,0.1,0.0001,1,10000000,0.1,...,,[SUI-USDC],SUI,USD,False,0.0001,SUI-USD,CBE,91721782.75,2023-05-17T15:54:00.704Z
8,USDT-USD,0.99991,0.00500070009801,79095447.05,32.1502359571439,0.01,0.00001,1,10000000,0.01,...,,[],USDT,USD,False,0.00001,USDT-USD,CBE,79088328.46,2023-01-01T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,DEXT-USD,0.2081,-2.62049602246139,30140.4,-28.19063774635002,0.1,0.0001,1,10000000,0.1,...,,[DEXT-USDC],DEXT,USD,False,0.0001,DEXT-USD,CBE,6272.22,2023-01-01T00:00:00Z
677,LSETH-USD,1893.59,-3.5698098987111,3.03268,4674.37027707808564,0.00001,0.01,1,10000000,0.00001,...,,[LSETH-USDC],LSETH,USD,False,0.01,LSETH-USD,CBE,5742.65,2023-01-01T00:00:00Z
684,STG-USD,0.1921,-1.78936605316973,24408.1,-19.96740737826131,0.1,0.0001,1,10000000,0.1,...,,[STG-USDC],STG,USD,False,0.0001,STG-USD,CBE,4688.8,2024-07-09T19:03:00.025Z
692,MSOL-USD,184.87,-0.73560996563574,15.41,-65.68310878521323,0.001,0.01,1,10000000,0.001,...,,[MSOL-USDC],MSOL,USD,False,0.01,MSOL-USD,CBE,2848.85,2023-01-01T00:00:00Z


## Step 2: Map Coinbase Products to CoinGecko
This step maps the product data fetched from Coinbase to CoinGecko data using their respective APIs.

In [ ]:
import requests
import source.code.coingecko as cg

def coinbase_products_to_coingecko(prod, limit=10):
    # change product base name to slug for coingecko
    p_ids = prod['base_name'].tolist()
    p_ids = [p_id.lower().replace(' ', '-') for p_id in p_ids]

    error_log = []
    coin_gecko_products = []

    for i, p_id in enumerate(p_ids):
        print(f"Processing {i+1}/{len(p_ids)}: {p_id}")
        try:
            coin_gecko_products.append(cg.retry_get_product_by_id(p_id))
        except ValueError as e:
            error_log.append(p_id)

        if i == limit:
            break
    return coin_gecko_products, error_log

cgp, e_log = coinbase_products_to_coingecko(product, limit=10)

Processing 1/297: bitcoin
Processing 2/297: ethereum
Processing 3/297: xrp
Processing 4/297: sui
Processing 5/297: tether
Processing 6/297: solana
Processing 7/297: litecoin
Processing 8/297: bonk
Processing 9/297: official-trump
Processing 10/297: dogecoin
Processing 11/297: stellar-lumens


## Step 3: Convert CoinGecko Data to DataFrame
The CoinGecko data is converted into a Pandas DataFrame for further analysis.

In [4]:
cgp = pd.DataFrame(cgp)

## Step 4: Extract and Process Categories
This step extracts the categories from the CoinGecko data and processes them for further use.

In [6]:
categories = cgp[['id', 'categories']]
categories = categories.explode('categories')
categories

,id,categories
0,bitcoin,Smart Contract Platform
0,bitcoin,Layer 1 (L1)
0,bitcoin,FTX Holdings
0,bitcoin,Proof of Work (PoW)
0,bitcoin,Bitcoin Ecosystem
...,...,...
8,dogecoin,Proof of Work (PoW)
8,dogecoin,GMCI Meme Index
8,dogecoin,GMCI 30 Index
8,dogecoin,GMCI Index
